Training

In [ ]:
#installation of packages:
!pip install autotrain-advanced
!pip install huggingface_hub
!pip install transformers
!pip install datasets
!pip install vllm

In [ ]:
!autotrain setup --update-torch

In [ ]:
#HuggingFace credentials:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from datasets import load_dataset
import csv

def reformat_dataset(dataset_name, output_file):
    # Load the dataset
    dataset = load_dataset(dataset_name)

    # Open the output CSV file in write mode
    with open(output_file, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)

        # Write the header row
        writer.writerow(["text"])

        # Iterate through each example in the dataset
        for example in dataset['train']:
            # Extract the components from the example
            instruction = example['instruction']
            response = example['response']
            context = example['context']

            # Write the formatted entry to the CSV file
            writer.writerow([f"### Human: {context} {instruction} ### Assistant: {response}"])

# Example usage:
reformat_dataset("ceadar-ie/FinTalk-19k", "dataset.csv")

In [ ]:
#Fine-Tune
!autotrain llm --train --project-name 'FinGyan' --model AdaptLLM/finance-chat --data-path 'data_path' --text_column 'text' --use-peft --lr 0.0002 --auto_find_batch_size --epochs 5 --trainer sft --quantization int8 --mp fp16 --model_max_length 512 --block_size 512 > training.log

> INFO    Running LLM
> INFO    Params: Namespace(version=False, text_column='text', rejected_text_column='rejected', prompt_text_column='prompt', model_ref=None, warmup_ratio=0.1, optimizer='adamw_torch', scheduler='linear', weight_decay=0.0, max_grad_norm=1.0, add_eos_token=False, block_size=512, peft=True, lora_r=16, lora_alpha=32, lora_dropout=0.05, logging_steps=-1, evaluation_strategy='epoch', save_total_limit=1, save_strategy='epoch', auto_find_batch_size=True, mixed_precision='fp16', quantization='int8', model_max_length=512, trainer='sft', target_modules=None, merge_adapter=False, use_flash_attention_2=False, dpo_beta=0.1, apply_chat_template=False, padding=None, train=True, deploy=False, inference=False, username=None, backend='local-cli', token=None, repo_id=None, push_to_hub=False, model='AdaptLLM/finance-chat', project_name='FinGyan', seed=42, epochs=5, gradient_accumulation=1, disable_gradient_checkpointing=False, lr=0.0002, log='none', data_path='/content/Dataset', train

Inference

In [ ]:
import json
from vllm import LLM, SamplingParams

# Load the JSON configuration
with open(r"/kaggle/input/finqa/other/finqa/1/adapter_config.json", 'r') as f:
    config = json.load(f)
    model_id = config['base_model_name_or_path']

# Initialize the LLM with adjusted parameters
llm = LLM(
    model=model_id,
    gpu_memory_utilization=1,
    enforce_eager=True,
    max_model_len=512
)

In [ ]:
prompts = ['How to select the stocks for the portfolio ?', 'what is portfolio optimization ?']

sampling_params = SamplingParams(temperature=0.5, max_tokens=1024)

outputs = llm.generate(prompts, sampling_params)

# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"\n\nPrompt: {prompt!r}, \n\nGenerated text: {generated_text!r}")